<a href="https://colab.research.google.com/github/aqeeelmirza/hello-world/blob/master/IMDB_Movie_Reviews_Large_Dataset_50k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ktrain


     |████████████████████████████████| 25.3MB 128kB/s 
     |████████████████████████████████| 983kB 47.4MB/s 
     |████████████████████████████████| 245kB 50.0MB/s 
     |████████████████████████████████| 890kB 52.3MB/s 
     |████████████████████████████████| 471kB 44.6MB/s 
     |████████████████████████████████| 3.0MB 50.6MB/s 
     |████████████████████████████████| 890kB 48.2MB/s 
     |████████████████████████████████| 1.1MB 52.7MB/s 
  Created wheel for ktrain: filename=ktrain-0.21.2-cp36-none-any.whl size=25269840 sha256=0f12364c72472c60eefca0be5e4bd001527a33ace851c5879987de2e4bc298b8
  Stored in directory: /root/.cache/pip/wheels/5a/16/d5/4673e3d2507440040ce1a144db9946b77d08a666e3dd25ddbc
  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp36-none-any.whl size=34145 sha256=ce1ebda026e0cceede449c50b4f364996be84592af2d17d626a48396b18299b8
  Stored in directory: /root/.cache/pip/wheels/66/f0/b1/748128b58562fc9e31b907bb5e2ab6a35eb37695e83911236b
  Created wheel for lan

In [2]:
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git


Cloning into 'IMDB-Movie-Reviews-Large-Dataset-50k'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import ktrain
from ktrain import text

In [4]:
data_train = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx' , dtype = str)
data_train.head()

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg


In [5]:
data_test = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx' , dtype = str)
data_test.head()

,Reviews,Sentiment
0,Who would have thought that a movie about a ma...,pos
1,After realizing what is going on around us ......,pos
2,I grew up watching the original Disney Cindere...,neg
3,David Mamet wrote the screenplay and made his ...,pos
4,"Admittedly, I didn't have high expectations of...",neg


In [6]:
import pandas as pd
(x_train, y_train), (x_test, y_test), preprocess = text.texts_from_df(train_df = data_train,
                                                                      text_column = 'Reviews',
                                                                      label_columns = 'Sentiment',
                                                                      val_df = data_test,
                                                                      maxlen = 512,
                                                                      preprocess_mode = 'bert')

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [7]:
x_train[0].shape

(25000, 512)

In [8]:
model = text.text_classifier(name='bert',
                             train_data = (x_train, y_train),
                             preproc = preprocess)

Is Multi-Label? False
maxlen is 512
done.


In [9]:
learner = ktrain.get_learner(model = model,
                             train_data = (x_train, y_train),
                             val_data = (x_test, y_test),
                             batch_size = 6)

In [ ]:
#this might take days to run
# learner.lr_find()
# learner.lr_plot()
# Optimal learning rate of this model is 2e-5

In [10]:
learner.fit_onecycle(lr = 2e-5, epochs = 3)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/3
4167/4167 [==============================] - 5474s 1s/step - loss: 0.2664 - accuracy: 0.8896 - val_loss: 0.1535 - val_accuracy: 0.9499
Epoch 2/3
4167/4167 [==============================] - 5472s 1s/step - loss: 0.1608 - accuracy: 0.9396 - val_loss: 0.0531 - val_accuracy: 0.9850
Epoch 3/3
4167/4167 [==============================] - 5473s 1s/step - loss: 0.0545 - accuracy: 0.9832 - val_loss: 0.0149 - val_accuracy: 0.9968


In [11]:
learner

In [13]:
predictor = ktrain.get_predictor(learner.model, preprocess)

In [14]:
data = ['this movie was horrible. the plot was really borring',
        'this movie is really sucked and acting is very bad',
        'what a beautiful movie and acting was great']

In [15]:
predictor.predict(data)

['neg', 'neg', 'pos']

In [16]:
predictor.save('/content/bert')

In [17]:
!zip -r /content/bert.zip /content/bert

  adding: content/bert/ (stored 0%)
  adding: content/bert/tf_model.h5 (deflated 10%)
  adding: content/bert/tf_model.preproc (deflated 52%)
